# Lets Use Libraries!

The objective of the last two days has been to familiarize you with the fundamentals of neural networks: terminology, structure of networks, forward propagation, error/cost functions, backpropagation, epochs, and gradient descent. We have tried to reinforce these topics by requiring to you code some of the simplest neural networks by hand including Perceptrons (single node neural networks) and Multi-Layer Perceptrons also known as Feed-Forward Neural Networks. Continuing to do things by hand would not be the best use of our limited time. You're ready to graduate from doing things by hand and start using some powerful libraries to build cutting-edge predictive models. 

# Keras

> "Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research. Use Keras if you need a deep learning library that:

> Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).
Supports both convolutional networks and recurrent networks, as well as combinations of the two.
Runs seamlessly on CPU and GPU." 

## Installation

The Keras API is particularly straightforward and it already comes pre-installed on Google Colab! 

<img src="http://www.ryanleeallred.com/wp-content/uploads/2019/04/pip-freeze-keras.png" width="300">

If you're not on Google Colab you'll need to install one of the "backend" engines that Keras runs on top of. I recommend Tensorflow:

> `pip install tensorflow`

Google Colab does not have the latest Tensorflow 2.0 installation, so you'll need to upgrade to that if you want to experiment with it. However Tensorflow 2.0 was just released last month and is still in "alpha" so if you **really** want to use the latest and greatest be prepared for odd bugs that you don't have control over every once in a while. <https://www.tensorflow.org/install/>

## Our First Keras Model - Perceptron, Batch epochs

1) Load Data

2) Define Model

3) Compile Model

4) Fit Model

5) Evaluate Model

### Load Data

Our life is going to be easier if our data is already cleaned up and numeric, so lets use this dataset from Jason Brownlee that is already numeric and has no column headers so we'll need to slice off the last column of data to act as our y values.

In [2]:
import pandas as pd

In [6]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
dataset = pd.read_csv(url, header=None)

In [7]:
dataset.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
X = dataset.values[:,0:8]
print(X.shape)

(768, 8)


In [25]:
Y = dataset.values[:,-1]
print(Y.shape)

(768,)


### Define Model

In [16]:
from tensorflow.keras.model import Sequential

ModuleNotFoundError: No module named 'tensorflow.keras.model'

In [17]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(42)

I'll instantiate my model as a "sequential" model. This just means that I'm going to tell Keras what my model's architecture should be one layer at a time.

In [18]:
# https://keras.io/getting-started/sequential-model-guide/
model = Sequential()

Adding a "Dense" layer to our model is how we add "vanilla" perceptron-based layers to our neural network. These are also called "fully-connected" or "densely-connected" layers. They're used as a layer type in lots of other Neural Net Architectures but they're not referred to as perceptrons or multi-layer perceptrons very often in those situations even though that's what they are.

 > ["Just your regular densely-connected NN layer."](https://keras.io/layers/core/)
 
 The first argument is how many neurons we want to have in that layer. To create a perceptron model we will just set it to 1. We will tell it that there will be 8 inputs coming into this layer from our dataset and set it to use the sigmoid activation function.

In [19]:
model.add(Dense(1, input_dim=8, activation="sigmoid"))

Instructions for updating:
Colocations handled automatically by placer.


### Compile Model
Using binary_crossentropy as the loss function here is just telling keras that I'm doing binary classification so that it can use the appropriate loss function accordingly. If we were predicting non-binary categories we might assign something like `categorical_crossentropy`. We're also telling keras that we want it to report model accuracy as our main error metric for each epoch. We will also be able to see the overall accuracy once the model has finished training.

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit Model

Lets train it up! `model.fit()` has a `batch_size` parameter that we can use if we want to do mini-batch epochs, but since this tabular dataset is pretty small we're just going to delete that parameter. Keras' default `batch_size` is `None` so omiting it will tell Keras to do batch epochs. 

In [26]:
model.fit(X, Y, epochs=150)

Epoch 1/150
768/768 [==============================] - 0s 34us/step - loss: 3.0081 - acc: 0.6940
Epoch 2/150
768/768 [==============================] - 0s 33us/step - loss: 3.0071 - acc: 0.6914
Epoch 3/150
768/768 [==============================] - 0s 43us/step - loss: 3.0110 - acc: 0.6849
Epoch 4/150
768/768 [==============================] - 0s 35us/step - loss: 3.0115 - acc: 0.6849
Epoch 5/150
768/768 [==============================] - 0s 33us/step - loss: 3.0124 - acc: 0.6888
Epoch 6/150
768/768 [==============================] - 0s 47us/step - loss: 3.0085 - acc: 0.6901
Epoch 7/150
768/768 [==============================] - 0s 34us/step - loss: 3.0072 - acc: 0.6927
Epoch 8/150
768/768 [==============================] - 0s 44us/step - loss: 3.0095 - acc: 0.6823
Epoch 9/150
768/768 [==============================] - 0s 35us/step - loss: 3.0108 - acc: 0.6836
Epoch 10/150
768/768 [==============================] - 0s 39us/step - loss: 3.0053 - acc: 0.6849
Epoch 11/150
768/768 [=======

### Evaluate Model

In [24]:
scores = model.evaluate(X, y)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

768/768 [==============================] - 0s 61us/step
acc: 69.27083333333334


# Keras Perceptron Model in 4 lines of code:

In [30]:
model = Sequential()
model.add(Dense(1, input_dim=8, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=150)

Epoch 1/150
768/768 [==============================] - 0s 418us/step - loss: 8.9745 - acc: 0.3763
Epoch 2/150
768/768 [==============================] - 0s 33us/step - loss: 7.7985 - acc: 0.4102
Epoch 3/150
768/768 [==============================] - 0s 32us/step - loss: 6.2591 - acc: 0.4766
Epoch 4/150
768/768 [==============================] - 0s 41us/step - loss: 4.9841 - acc: 0.5417
Epoch 5/150
768/768 [==============================] - 0s 57us/step - loss: 4.2102 - acc: 0.6016
Epoch 6/150
768/768 [==============================] - 0s 45us/step - loss: 3.9298 - acc: 0.6250
Epoch 7/150
768/768 [==============================] - 0s 45us/step - loss: 3.8096 - acc: 0.6237
Epoch 8/150
768/768 [==============================] - 0s 39us/step - loss: 3.7029 - acc: 0.6263
Epoch 9/150
768/768 [==============================] - 0s 42us/step - loss: 3.6072 - acc: 0.6393
Epoch 10/150
768/768 [==============================] - 0s 42us/step - loss: 3.5437 - acc: 0.6393
Epoch 11/150
768/768 [======

In [31]:
# evaluate the model
scores = model.evaluate(X, Y)
print(f"{model.metrics_names[1]}: {scores[1]*100}")

768/768 [==============================] - 0s 113us/step
acc: 65.49479166666666


### Why are we getting such different results if we re-run the model?

<https://machinelearningmastery.com/randomness-in-machine-learning/>

# What architecture should we try?

In [36]:
model_improved = Sequential()

# Input + 1 First Hidden
model_improved.add(Dense(10, input_dim=8, activation='relu'))
# Hidden
model_improved.add(Dense(3, activation='sigmoid'))
# Output
model_improved.add(Dense(1, activation='sigmoid'))

model_improved.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])
model_improved.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 33        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 4         
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________


In [39]:
model_improved.fit(X, Y, epochs=2000)

Epoch 1/2000
768/768 [==============================] - 0s 51us/step - loss: 0.5776 - acc: 0.6641
Epoch 2/2000
768/768 [==============================] - 0s 50us/step - loss: 0.5804 - acc: 0.6654
Epoch 3/2000
768/768 [==============================] - 0s 43us/step - loss: 0.5779 - acc: 0.6706
Epoch 4/2000
768/768 [==============================] - 0s 53us/step - loss: 0.5831 - acc: 0.6602
Epoch 5/2000
768/768 [==============================] - 0s 45us/step - loss: 0.5764 - acc: 0.6641
Epoch 6/2000
768/768 [==============================] - 0s 42us/step - loss: 0.5791 - acc: 0.6654
Epoch 7/2000
768/768 [==============================] - 0s 39us/step - loss: 0.5824 - acc: 0.6680
Epoch 8/2000
768/768 [==============================] - 0s 46us/step - loss: 0.5776 - acc: 0.6628
Epoch 9/2000
768/768 [==============================] - 0s 45us/step - loss: 0.5778 - acc: 0.6667
Epoch 10/2000
768/768 [==============================] - 0s 37us/step - loss: 0.5774 - acc: 0.6667
Epoch 11/2000
768/7

# Activation Functions

What is an activation function and how does it work?

- Takes in a weighted sum of inputs + a bias from the previous layer and outputs an "activation" value.
- Based its inputs the neuron decides how 'activated' it should be. This can be thought of as the neuron deciding how strongly to fire. You can also think of it as if the neuron is deciding how much of the signal that it has received to pass onto the next layer. 
- Our choice of activation function does not only affect signal that is passed forward but also affects the backpropagation algorithm. It affects how we update weights in reverse order since activated weight/input sums become the inputs of the next layer. 


## Step Function

![Heaviside Step Function](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Dirac_distribution_CDF.svg/325px-Dirac_distribution_CDF.svg.png)

All or nothing, a little extreme, which is fine, but makes updating weights through backpropagation impossible. Why? remember that during backpropagation we use derivatives in order to determine how much to update or not update weights. What is the derivative of the step function?

## Linear Function

![Linear Function](http://www.roconnell.net/Parent%20function/linear.gif)

The linear function takes the opposite tact from the step function and passes the signal onto the next layer by a constant factor. There are problems with this but the biggest problems again lie in backpropagation. The derivative of any linear function is a horizontal line which would indicate that we should update all weights by a constant amount every time -which on balance wouldn't change the behavior of our network. Linear functions are typically only used for very simple tasks where interpretability is important, but if interpretability is your highest priority, you probably shouldn't be using neural networks in the first place.

## Sigmoid Function

![Sigmoid Function](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/480px-Logistic-curve.svg.png)

The sigmoid function works great as an activation function! it's continuously differentiable, its derivative doesn't have a constant slope, and having the higher slope in the middle pushes y value predictions towards extremes which is particularly useful for binary classification problems. I mean, this is why we use it as the squishifier in logistic regression as well. It constrains output, but over repeated epochs pushes predictions towards a strong binary prediction. 

What's the biggest problem with the sigmoid function? The fact that its slope gets pretty flat so quickly after its departure from zero. This means that updating weights based on its gradient really diminishes the size of our weight updates as our model gets more confident about its classifications. This is why even after so many iterations with our test score example we couldn't reach the levels of fit that our gradient descent based model could reach in just a few epochs.

## Tanh Function

![Tanh Function](http://mathworld.wolfram.com/images/interactive/TanhReal.gif)

What if the sigmoid function didn't get so flat quite as soon when moving away from zero and was a little bit steeper in the middle? That's basically the Tanh function. The Tanh function can actually be created by scaling the sigmoid function by 2 in the y dimension and subtracting 1 from all values. It has basically the same properties as the sigmoid, still struggles from diminishingly flat gradients as we move away from 0, but its derivative is higher around 0 causing weights to move to the extremes a little faster. 

## ReLU Function

![ReLU Function](https://cdn-images-1.medium.com/max/937/1*oePAhrm74RNnNEolprmTaQ.png)

ReLU stands for Rectified Linear Units it is by far the most commonly used activation function in modern neural networks. It doesn't activate neurons that are being passed a negative signal and passes on positive signals. Think about why this might be useful. Remember how a lot of our initial weights got set to negative numbers by chance? This would have dealt with those negative weights a lot faster than the sigmoid function updating. What does the derivative of this function look like? It looks like the step function! This means that not all neurons are activated. With sigmoid basically all of our neurons are passing some amount of signal even if it's small making it hard for the network to differentiate important and less important connections. ReLU turns off a portion of our less important neurons which decreases computational load, but also helps the network learn what the most important connections are faster. 

What's the problem with relu? Well the left half of its derivative function shows that for neurons that are initialized with weights that cause them to have no activation, our gradient will not update those neuron's weights, this can lead to dead neurons that never fire and whose weights never get updated. We would probably want to update the weights of neurons that didn't fire even if it's just by a little bit in case we got unlucky with our initial weights and want to give those neurons a chance of turning back on in the future.

## Leaky ReLU

![Leaky ReLU](https://cdn-images-1.medium.com/max/1600/1*ypsvQH7kvtI2BhzR2eT_Sw.png)

Leaky ReLU accomplishes exactly that! it avoids having a gradient of 0 on the left side of its derivative function. This means that even "dead" neurons have a chance of being revived over enough iterations. In some specifications the slope of the leaky left-hand side can also be experimented with as a hyperparameter of the model!

## Softmax Function

![Softmax Function](https://cdn-images-1.medium.com/max/800/1*670CdxchunD-yAuUWdI7Bw.png)

Like the sigmoid function but more useful for multi-class classification problems. The softmax function can take any set of inputs and translate them into probabilities that sum up to 1. This means that we can throw any list of outputs at it and it will translate them into probabilities, this is extremely useful for multi-class classification problems. Like MNIST for example...

## Major takeaways

- ReLU is generally better at obtaining the optimal model fit.
- Sigmoid and its derivatives are usually better at classification problems.
- Softmax for multi-class classification problems. 

You'll typically see ReLU used for all initial layers and then the final layer being sigmoid or softmax for classification problems. But you can experiment and tune these selections as hyperparameters as well!

## MNIST with Keras 

### This will be a good chance to bring up dropout regularization. :)

In [77]:
### Let's do it!
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np


In [69]:
batch_size = 64
num_class = 10
epochs = 150

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [70]:
X_train.shape

(60000, 28, 28)

In [71]:
# Hyperparameters
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

In [72]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [73]:
y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

In [74]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [75]:
mnist_model = Sequential()

# Hidden
mnist_model.add(Dense(16, input_dim=784, activation='relu'))
mnist_model.add(Dense(16, activation='relu'))
# Output Layer
mnist_model.add(Dense(10, activation='softmax'))

mnist_model.compile(loss='categorical_crossentropy',
                    optimizer='adam', 
                    metrics=['accuracy'])
mnist_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 16)                12560     
_________________________________________________________________
dense_17 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_18 (Dense)             (None, 10)                170       
Total params: 13,002
Trainable params: 13,002
Non-trainable params: 0
_________________________________________________________________


In [76]:
mnist_model.fit(X_train, y_train, epochs=epochs, validation_split=.1, verbose=0)
scores = mnist_model.evaluate(X_Test, y_test)

NameError: name 'X_Test' is not defined

## What if we use dropout techniques to prevent overfitting? How does that affect our model?

![Regularization](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Regularization.svg/354px-Regularization.svg.png)

In [ ]:
### Try it with dropout